# CAFEf6_ETBFcalc2D_writeTUzarr

**Date:** <br>
16 March 2022 <br>
**Background:** <br>
Issue - https://github.com/Thomas-Moore-Creative/NCI-CAFE-ARD/issues/2 <br>
**Author(s):**<br>
Thomas Moore<br>

## We are using NCI OOD as platform for data processing
### OOD documentation
https://opus.nci.org.au/display/DAE/Setting+up+a+Dask+Cluster+on+OOD

In [1]:
Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

In [2]:
import xarray as xr
import numpy as np
import xrft
import xesmf as xe
import scipy
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os
import re
import cartopy.crs as ccrs
import proplot as pplt
from rechunker import rechunk
%config Completer.use_jedi = False

## import helper

In [3]:
import importlib.util
spec = importlib.util.spec_from_file_location("helper", "/g/data/v14/tm4888/code/helper-py/helper_tools.py")
helper = importlib.util.module_from_spec(spec)
spec.loader.exec_module(helper)

## OOD cluster

In [4]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,processes=1,memory="47GB",walltime='03:00:00')
client = Client(cluster)
cluster.scale(cores=24)

/g/data/v14/tm4888/miniconda3/envs/pangeo_hpc/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [5]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.136:36165,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# choose to preserve attributes

In [6]:
xr.set_options(keep_attrs=True)

In [30]:
path_in = '/g/data/xv83/dcfp/CAFE-f6/c5-d60-pX-f6-20151101/'
path_out = '/g/data/xv83/users/tm4888/data/CAFE/hindcast_f6/'

# load CAFEf6 Tgrid and Ugrid zarr collections

In [8]:
ds_CAFEf6_Tgrid = xr.open_zarr(path_out + 'CAFEf6_Tgrid_reduced_ocean_month.zarr',consolidated=True)
ds_CAFEf6_Ugrid = xr.open_zarr(path_out + 'CAFEf6_Ugrid_reduced_ocean_month.zarr',consolidated=True)

# load CAFE grid info // use `layer_depth_t` for depth weights

In [9]:
grid_spec = xr.open_dataset('/g/data/xv83/users/tm4888/data/CAFE/grid_spec.auscom.20110118.nc')
CAFE_ocean_grid_info = xr.open_zarr('/g/data/xv83/users/tm4888/data/CAFE/CAFE60_ocean_grid_info.zarr',consolidated=True)
depth_weights = CAFE_ocean_grid_info.layer_depth_t

In [10]:
#T-grid
volume_weights_t = CAFE_ocean_grid_info.volume_t.chunk({'st_ocean': 50,'yt_ocean' : 300, 'xt_ocean' : 360})
area_weights_t = CAFE_ocean_grid_info.area_t
volume_dim_t={'st_ocean','yt_ocean','xt_ocean'}
area_dim_t={'yt_ocean','xt_ocean'}
#U-grid
volume_weights_u = CAFE_ocean_grid_info.volume_u.chunk({'st_ocean': 50,'yu_ocean' : 300, 'xu_ocean' : 360})
area_weights_u = CAFE_ocean_grid_info.area_u
volume_dim_u={'st_ocean','yu_ocean','xu_ocean'}
area_dim_u={'yu_ocean','xu_ocean'}

#calc some grid metric missing
dxt = grid_spec.ds_01_11_T + grid_spec.ds_11_21_T
dyt = grid_spec.ds_10_11_T + grid_spec.ds_11_12_T
dxu = grid_spec.ds_00_10_T + grid_spec.ds_10_20_T
dyu = grid_spec.ds_00_01_T + grid_spec.ds_01_02_T

# slice out the simple 2D variables

In [11]:
#T-grid
sst = ds_CAFEf6_Tgrid.sst
temp50 = ds_CAFEf6_Tgrid.temp.sel(st_ocean=50,method='nearest').rename('temp50')
temp100 = ds_CAFEf6_Tgrid.temp.sel(st_ocean=100,method='nearest').rename('temp100')
temp200 = ds_CAFEf6_Tgrid.temp.sel(st_ocean=200,method='nearest').rename('temp200')
temp500 = ds_CAFEf6_Tgrid.temp.sel(st_ocean=500,method='nearest').rename('temp500')
sss = ds_CAFEf6_Tgrid.salt.isel(st_ocean=0).rename('sss')

In [12]:
#U-grid
u100 = ds_CAFEf6_Ugrid.u.sel(st_ocean=100,method='nearest').rename('u100')
v100 = ds_CAFEf6_Ugrid.v.sel(st_ocean=100,method='nearest').rename('v100')

# Calculate all the extra quantaties from 3D T & U grid data // 
### D20, MLD, OHC, EKE, U/V100_300

#### functions

In [13]:
def calc_EKE(u, v, time_name = 'time'):
    '''
    Author1 = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}
    u,v are x and y currents as an xarray data array
    '''
    u_mean = u.mean(time_name)
    v_mean = v.mean(time_name)
    MKE = 0.5*(u_mean**2 + v_mean**2).rename('MKE') # currents
    EKE = ( 0.5 * ((u-u_mean)**2 + (v-v_mean)**2) ).rename('EKE') # eddies
    return EKE, MKE

In [14]:
def weighted_ocean_mean(ds,weights=None,dim=None,sel_dict=None):
    """
    weighted_ocean_mean
    Returns: ds_weighted
    Defaults: no spatial and time selection, no weights, no selected dims
    Author: Thomas Moore (based on Dougie Squire code)
    Date created: 11/02/2020

    Assumptions:
    Dataset or Dataarray = ds
    Use:
    Limitations:
    """

    if weights is None:
        return ds.sel(sel_dict).mean(dim)
    else:
        #weights = (0 * da + 1) * weights
        #return (da * weights).sum(dim) / weights.sum(dim)
        return (ds.sel(sel_dict) * weights).sum(dim,skipna=True) / weights.where(ds.sel(sel_dict).notnull()).sum(dim,skipna=True)

### U-grid depth weighted means

In [15]:
u100_300 = helper.weighted_ocean_mean(ds_CAFEf6_Ugrid.u,weights=depth_weights,dim='st_ocean',sel_dict={'st_ocean':slice(100,300)})
v100_300 = helper.weighted_ocean_mean(ds_CAFEf6_Ugrid.v,weights=depth_weights,dim='st_ocean',sel_dict={'st_ocean':slice(100,300)})

In [16]:
u100_300 = u100_300.rename('u100_300')
v100_300 = v100_300.rename('v100_300')

In [17]:
u100_300.attrs['long_name'] = 'mean U current 100-300m '
u100_300.attrs['units'] = 'm/s'
timestamp = datetime.today().strftime('%Y-%m-%d')
u100_300.attrs['History'] = 'Created by Thomas Moore on ' + timestamp + ' using a "depth weighted mean" approach'

In [18]:
v100_300.attrs['long_name'] = 'mean V current 100-300m '
v100_300.attrs['units'] = 'm/s'
timestamp = datetime.today().strftime('%Y-%m-%d')
v100_300.attrs['History'] = 'Created by Thomas Moore on ' + timestamp + ' using a "depth weighted mean" approach'

### EKE300 & 2000

In [19]:
%%time
[EKE, MKE] = calc_EKE(ds_CAFEf6_Ugrid.u, ds_CAFEf6_Ugrid.v, time_name='time')

CPU times: user 41.4 ms, sys: 4.54 ms, total: 45.9 ms
Wall time: 43.3 ms


In [20]:
eke300 = (EKE * depth_weights).sel(st_ocean=slice(0,300)).sum('st_ocean',skipna=True)
eke300 = eke300.where(eke300 != 0)

In [21]:
eke2000 = (EKE * depth_weights).sel(st_ocean=slice(0,2000)).sum('st_ocean',skipna=True)
eke2000 = eke2000.where(eke2000 != 0)

In [22]:
eke300 = eke300.rename('eke300')
eke2000 = eke2000.rename('eke2000')

In [23]:
eke300.attrs['long_name'] = 'EKE intgral 0-300m '
eke300.attrs['units'] = 'm^2 / s^2'
timestamp = datetime.today().strftime('%Y-%m-%d')
eke300.attrs['History'] = 'Created by Thomas Moore on ' + timestamp + ' using a "weighted depth sum" approach'

In [24]:
eke2000.attrs['long_name'] = 'EKE intgral 0-2000m '
eke2000.attrs['units'] = 'm^2 / s^2'
timestamp = datetime.today().strftime('%Y-%m-%d')
eke2000.attrs['History'] = 'Created by Thomas Moore on ' + timestamp + ' using a "weighted depth sum" approach'

# D20

In [25]:
def isosurface(da, coord, target, interp_flag = False):
    """
        Returns the values of a coordinate in the input array where the input array values equals \
                a prescribed target. E.g. returns the depth of the 20 degC isotherm. Returns nans for all \
                points in input array where isosurface is not defined. If 
        
        | Author: Thomas Moore and Dougie Squire
        | Date: 02/10/2018
        
        Parameters
        ----------
        da : xarray DataArray
            Array of values to be isosurfaced
        coord : str
            Name of coordinate to contruct isosurface about
        target : value
            Isosurface value
            
        Returns
        -------
        isosurface : xarray DataArray
            Values of coord where da is closest to target. If multiple occurences of target occur \
                    along coord, only the maximum value of coord is returned
            
        Examples
        --------
        >>> A = xr.DataArray(np.random.normal(size=(5,5)), 
        ...                  coords=[('x', np.arange(5)), ('y', np.arange(5))])
        >>> isosurface(A, coord='x', target=0)
        >>> doppyo.utils.isosurface(A, coord='x', target=0)
        <xarray.DataArray 'isosurface' (y: 5)>
        array([ 4.,  1., nan,  3.,  4.])
        Coordinates:
          * y        (y) int64 0 1 2 3 4
  
        Notes
        -----------
        If multiple occurences of target occur along coord, only the maximum value of coord is\
                returned
        
        To do
        
        - The current version includes no interpolation between grid spacing. This should be added as \
                an option in the future
    """
    
    # Find isosurface -----
    if interp_flag == False:
        mask = da > target
        da_mask = mask * da[coord]
        isosurface = da_mask.max(coord)
        

    return isosurface.where(da.max(dim=coord) > target).rename('isosurface')

def isotherm_depth(temp, target_temp=20, depth_name=None):
    """ 
        Returns the depth of an isotherm given a target temperature. If no temperatures in the column
        exceed the target temperature, a nan is returned at that point
        
        | Author: Thomas Moore
        | Date: 02/10/2018
        
        Parameters
        ----------
        temp : xarray DataArray
            Array containing values of temperature with at least a depth dimension
        target_temp : value, optional
            Value of temperature used to compute isotherm depth. Default value is 20 degC
        depth_name : str, optional
            Name of depth coordinate. If None, doppyo will attempt to determine depth_name \
                    automatically
            
        Returns
        -------
        isotherm_depth : xarray DataArray
            Array containing the depth of the requested isotherm
        Examples
        --------
        >>> temp = xr.DataArray(20 + np.random.normal(scale=5, size=(4,4,10)), 
        ...                     coords=[('lat', np.arange(-90,90,45)), ('lon', np.arange(0,360,90)), 
        ...                             ('depth', np.arange(2000,0,-200))])
        >>> doppyo.diagnostic.isotherm_depth(temp)
        <xarray.DataArray 'isosurface' (lat: 4, lon: 4)>
        array([[ 400., 1600., 2000.,  800.],
               [1800., 2000., 1800., 2000.],
               [2000., 2000., 2000., 1600.],
               [1400., 2000., 2000., 2000.]])
        Coordinates:
          * lat      (lat) int64 -90 -45 0 45
          * lon      (lon) int64 0 90 180 270
        
        Notes
        -----------
        | All input array coordinates must follow standard naming (see ``doppyo.utils.get_lat_name()``, \
                ``doppyo.utils.get_lon_name()``, etc)
        | If multiple occurences of target occur along the depth coordinate, only the maximum value of \
                coord is returned
        | The current version includes no interpolation between grid spacing. This should be added as \
                an option in the future
    """

    if depth_name is None:
        depth_name = helper.get_depth_name(temp)

    return isosurface(temp, coord=depth_name, target=target_temp).rename('isotherm_depth')

In [26]:
D20 = isotherm_depth(ds_CAFEf6_Tgrid.temp).rename('D20')

In [27]:
D20.attrs['long_name'] = 'depth of the 20degree isotherm'
D20.attrs['units'] = 'm'
timestamp = datetime.today().strftime('%Y-%m-%d')
D20.attrs['History'] = 'Created by Thomas Moore on ' + timestamp + ' using a "where max" approach'

# OHC200 & 300

### OHC calculation - $per\hspace{5mm}Taley\hspace{1mm}(2018)$

#### $OHC = \rho * c_p \int\limits_{h_{1}}^{h_{2}}T(z)dz $
#### The density of seawater can be approximated as 1025 $kg/m^3$ and the specific heat can be approximated as 3850 $J/(kg C)$
#### The above simplifies $\rho$ and $c_p$ as being constant with depth
#### For CAFEf6 we have and are using $potential\,temperature$ 
#### Units should end up being $J/m^2$

In [28]:
def get_OHC_viaWeightedSum(temp_da, h, depth_dim_name = 'st_ocean'):
    #define "constants" - note this calculation simplifies ρ and cp as constant with depth 
    ρ =  1025.0
    cp = 3850.0
    OHC = (temp_da * depth_weights).sel({depth_dim_name:slice(0,h)}).sum(depth_dim_name,skipna=True)*ρ*cp
    OHC = OHC.where(OHC != 0)
    OHC = OHC.rename('hc' + str(h))
    OHC.attrs['long_name'] = 'ocean heat content'
    OHC.attrs['units'] = 'J per square metre'
    timestamp = datetime.today().strftime('%Y-%m-%d')
    OHC.attrs['History'] = 'Created by Thomas Moore on ' + timestamp + ' using a depth weighted sum approach and temperature in degC'
    return OHC

In [29]:
hc200 = get_OHC_viaWeightedSum(ds_CAFEf6_Tgrid.temp, 200)
hc300 = get_OHC_viaWeightedSum(ds_CAFEf6_Tgrid.temp, 300)

# MLD

### using daily CAFEf6 data

In [31]:
ds_daily = xr.open_zarr(path_in+'ocean_daily.zarr.zip',consolidated=True)

In [32]:
mld = ds_daily.mld.mean('ensemble')
mld = mld.resample(time='1M').mean()

### crop time

In [33]:
mld = mld.isel(time=slice(0,37))

In [34]:
#brute force matching of time
mld['time']=ds_CAFEf6_Tgrid['time']

#### Attributes

In [35]:
timestamp = datetime.today().strftime('%Y-%m-%d')
mld.attrs['History'] = 'Created by Thomas Moore on ' + timestamp + ' using a resampled daily CAFEf6 data'

# merge into 2D Tgrid and Ugrid files

In [41]:
ds_CAFEf6_Tgrid2D = xr.merge([sst,sss,temp50,temp100,temp200,temp500,D20,mld,hc200,hc300],compat='override').drop('st_ocean')

In [42]:
ds_CAFEf6_Ugrid2D = xr.merge([u100,v100,u100_300,v100_300,eke300,eke2000],compat='override').drop('st_ocean')

# write out to zarr

In [43]:
ds_CAFEf6_Tgrid2D.nbytes/1e9

0.209094464

In [44]:
ds_CAFEf6_Ugrid2D.nbytes/1e9

0.161142464

In [45]:
%%time
ds_CAFEf6_Tgrid2D.to_zarr(path_out+'CAFEf6_Tgrid_2D.zarr',consolidated=True)

CPU times: user 12.8 s, sys: 1.73 s, total: 14.5 s
Wall time: 2min 22s


In [46]:
%%time
ds_CAFEf6_Ugrid2D.to_zarr(path_out+'CAFEf6_Ugrid_2D.zarr',consolidated=True)

CPU times: user 1.59 s, sys: 167 ms, total: 1.76 s
Wall time: 4.02 s


# $ The\ End$

# Break glass in case of emergency
# $\Downarrow$

In [ ]:
client.restart()

In [47]:
client.shutdown()

In [ ]:
client.restart()